In [8]:
%pwd

'/content'

In [9]:
%ls

drive/  sample_data/


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
%ls /content/gdrive/MyDrive/developments/Final_Year_Projects/ACA_LLM_KDD-24/dataAQA/

pid_to_title_abs_new.json  qa_train.txt  qa_valid_wo_ans.txt  result.txt


In [14]:
%cd /content/gdrive/MyDrive/developments/Final_Year_Projects/ACA_LLM_KDD-24/dataAQA/

/content/gdrive/MyDrive/developments/Final_Year_Projects/ACA_LLM_KDD-24/dataAQA


In [15]:
%ls

pid_to_title_abs_new.json  qa_train.txt  qa_valid_wo_ans.txt  result.txt


In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import json

# Load the tokenizer and model from the Hugging Face library
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Function to convert text to embeddings
def get_embeddings(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Load the data
with open('pid_to_title_abs_new.json') as f:
    paper_data = json.load(f)

# Convert paper titles and abstracts to embeddings
paper_embeddings = {}
for pid, info in paper_data.items():
    paper_embeddings[pid] = get_embeddings(info['title'] + ' ' + info['abstract'])

# Load the questions
with open('qa_train.txt') as f:
    questions = json.load(f)

# Convert questions to embeddings and find the most relevant papers
for question in questions:
    question_embedding = get_embeddings(question['question'] + ' ' + question['body'])
    similarities = {}
    for pid, embedding in paper_embeddings.items():
        cos_sim = torch.nn.functional.cosine_similarity(question_embedding, embedding, dim=0)
        similarities[pid] = cos_sim.item()

    # Sort by similarity
    relevant_pids = sorted(similarities, key=similarities.get, reverse=True)[:20]
    print(f"Question: {question['question']}")
    print(f"Top 20 relevant paper IDs: {relevant_pids}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
